World Development Indicators - worldbank.org <br/>
Fabio Cardoso - Dec/2021 <br/><br/>

Datasets: <br/><br/>

Country Indicators: <br/>
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators <br/>
Note: dataset updated with the topics presented in https://data.worldbank.org/indicator?tab=all - Utils notebook. <br/><br/>

Country Codes ISO 3166: <br/>
https://www.iban.com/country-codes <br/><br/>

Note: <br/>
For some orphans indicators, the topic 'private-sector' was assumed - Utils notebook. <br/>
. Firms visited or required meetings with tax officials (% of firms) <br/>
. Net ODA provided to the least developed countries (% of GNI) <br/>
. Net ODA provided, to the least developed countries (current US )−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡2015𝑈𝑆) <br/>
. Net ODA provided, total (current US$)

In [1]:
# Parameters

cloud = False

In [2]:
# Imports

import pandas as pd
import numpy as np
#import psycopg2
import zipfile
import matplotlib.pyplot as plt
import zlib
import pickle
from threading import Thread
from multiprocessing import Pool
import time
from datetime import datetime
import os

!pip install pyreadstat
import pyreadstat

!pip install unidecode
import unidecode

In [ ]:
# The code was removed by Watson Studio for sharing.

In [3]:
# Storage instantiation

if cloud:
    import types
    import pandas as pd
    import ibm_boto3
    from botocore.client import Config

    def __iter__(self): return 0

    cnx_fcsinsights = ibm_boto3.client(
        service_name = 's3',
        ibm_api_key_id = parm_ibm_api_key_id,
        ibm_service_instance_id = parm_ibm_service_instance_id,
        ibm_auth_endpoint = parm_ibm_auth_endpoint,
        config = Config(signature_version='oauth'),
        endpoint_url = parm_endpoint_url
    )

bkt_in = parm_bkt_in
bkt_out = parm_bkt_out

key_in = 'WDI-treated-step5.csv.gz'
key_out = 'WDI-treated-step5.sav'

In [ ]:
# Read data

if cloud:
    cnx_fcsinsights.download_file(Bucket=bkt_in, Key=key_in, Filename=key_in)
    df = pd.read_csv(key_in, compression='gzip')
else:
    df = pd.read_csv('F:/WDI Working Files/'+key_in, compression='gzip')

In [ ]:
#saving to sav

i=0
cols = df.columns
for col in cols:
    i+=1
    if i%25==0:
        print(i,end='\r')
    colx = col.replace(' ', '_').replace('.','_')
    df.rename(columns={col:colx},inplace=True)
    
pyreadstat.write_sav(df, 'F:/WDI Working Files/'+key_out)

In [6]:
# Updating with categories
# Source of categories: https://data.worldbank.org/indicator?tab=all (via developer tool)

key_in = 'WDI_csv.zip'
zf = zipfile.ZipFile('F:WDI Working Files/'+key_in)
df0 = pd.read_csv(zf.open('WDIData.csv'))

arqs = os.listdir('F:WDI Working Files/WDI Categories')

relations = []
for arq in arqs:
    if arq.startswith("WDI"):
        print(arq)
        df1=pd.read_excel('F:WDI Working Files/WDI Categories/'+arq, sheet_name='Data', skiprows=3)
        try:
            df2=pd.read_excel(arq, sheet_name='Data (2)')
            df1=pd.concat([df1,df2],axis=0)
        except: pass
        df1.drop_duplicates(subset=['Indicator Code'], inplace=True)
        for i in range(df1.shape[0]):
            relations.append([arq, df1.iloc[i]['Indicator Code']])
            
df_rel = pd.DataFrame(relations, columns=['Category','Indicator Code'])

df_rel['Category'] = df_rel['Category'].apply(lambda x: x.replace('WDI ','').replace('.xls',''))

print(df0.shape)
dfm = df0.merge(df_rel, how='left', on='Indicator Code')
print(dfm.shape)

not_assigned = dfm[dfm['Category'].isna()]['Indicator Name'].unique()
print('not assigned..')
print(not_assigned)

#these not assigned indicators will be assigned to 'private-sector'
dfm['Category'] = dfm[['Category','Indicator Name']].apply(lambda r: 'private-sector' 
                                                           if r['Indicator Name'] in not_assigned 
                                                           else r['Category'], axis=1)

yrs = [x for x in dfm.columns if x.isnumeric()]
dfm = dfm[['Country Name','Country Code','Indicator Name','Indicator Code', 'Category'] + yrs]
dfm.to_csv('F:/WDI Working Files/WDI_plus_category.csv.gz', compression='gzip')

#uma categoria pode estar associada a mais de um indicador
df_rel.groupby(by=['Indicator Code']).nunique().sort_values(by=['Category'],ascending=False)[:3]

WDI agriculture-and-rural-development.xls
WDI aid-effectiveness.xls
WDI climate-change.xls
WDI economy-and-growth.xls
WDI education.xls
WDI energy-and-mining.xls
WDI environment.xls
WDI external-debt.xls
WDI financial-sector.xls
WDI gender.xls
WDI health.xls
WDI infrastructure.xls
WDI poverty.xls
WDI private-sector.xls
WDI public-sector.xls
WDI science-and-technology.xls
WDI social-development.xls
WDI social-protection-and-labor.xls
WDI trade.xls
WDI urban-development.xls
(383838, 66)
(551152, 67)
not assigned..
['Firms visited or required meetings with tax officials (% of firms)'
 'Net ODA provided to the least developed countries (% of GNI)'
 'Net ODA provided, to the least developed countries (current US$)'
 'Net ODA provided, total (% of GNI)'
 'Net ODA provided, total (constant 2015 US$)'
 'Net ODA provided, total (current US$)']


,Category
Indicator Code,
SE.ENR.PRSC.FM.ZS,5
SL.UEM.TOTL.MA.ZS,4
SL.UEM.TOTL.FE.ZS,4


In [64]:
# Salvando arquivo para virar tabela no sqlquery

df = pd.read_csv('F:/WDI Working Files/WDI-treated-step6 Kmeans General.csv')

In [66]:
# Remove spaces from column names and save as parquet

df.drop(columns={'Unnamed: 0'},inplace=True)

def remove_space_from_cols_names(df):
    rens = dict()
    for col in df.columns:
        col1 = col.lower() #lowercase
        col2 = col1.replace(" ", "_") #remove espaço
        col3 = unidecode.unidecode(col2) #remove acento
        rens.update({col:col2})
    return df.rename(columns=rens)

df = remove_space_from_cols_names(df)
df.rename(columns={'_clusterkm':'cluster_km_general'}, inplace=True)
df.to_parquet('F:/WDI Working Files/wdi_clusterkm_general.parquet', index=False)

In [10]:
## Generating Countries List Codes to be used in Cognos as dimension

import requests
import pandas as pd

url = 'https://www.iban.com/country-codes'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]
df.to_excel('F:/WDI Working Files/country_codes_iban.xlsx')